## Taller (Detección)
- Realizar un modelo para la localización de objetos (Regresión de las coordenadas de los Bounding Box)

Pasos:
Crear matriz de etiquetas ( )
Graficar etiquetas en imágenes:
Las etiquetas están dadas por
dos puntos
• Inicial (x_top, y_top)
• Final (x_bottom, y_bottom)
Generar Etiquetas:
Normalizar por el tamaño de
la imagen, de esta forma el
modelo predice valores entre
0-1 por cada coordenada
[x
...
w
,
y...
h ]

2. Modelo: Extractor de características
(Convoluciones) + Cabeza (regresión de 4
variables)
3. Entrenar con función de pérdida MSE (Mean
Squared Error)
Leer el archivo en formato COCO
("bbox" : [x,y,width,height])
2. Implementar una función para
calcular el IoU de dos rectángulos
3. Determinar los rectángulos con
mayor IOU

Este notebook entrena un modelo basado en `ResNet18` para detectar la ubicación de aviones en imágenes. Utiliza coordenadas corregidas de los bounding boxes, data augmentation y entrenamiento sobre imágenes redimensionadas. Se calcula el IoU promedio como métrica de desempeño.

In [1]:
import zipfile
import os

# Ruta al archivo ZIP
zip_path = 'data/airplanes.zip'

# Ruta del directorio de destino
extract_to = 'data/airplanes'

# Crear el directorio si no existe
os.makedirs(extract_to, exist_ok=True)

# Descomprimir el ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Descomprimido en: {extract_to}")

Descomprimido en: data/airplanes


In [2]:
import torch

# --- Selección dinámica del dispositivo ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Usando GPU NVIDIA")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Usando GPU Apple Silicon (MPS)")
else:
    device = torch.device("cpu")
    print("Usando CPU")

Usando GPU Apple Silicon (MPS)


1. Leer el archivo de etiquetas

El archivo Airplanes.csv tiene la información de los bounding boxes (en formato COCO: [x, y, width, height]). Empezaremos leyendo ese archivo y asociándolo con las imágenes correspondientes.

In [3]:
import pandas as pd

# Leer el archivo CSV con las etiquetas
df = pd.read_csv('data/Airplanes.csv')
print(df.columns)

# Mostrar algunas filas para entender la estructura
df.head()

Index(['Unnamed: 0', 'Image', 'x_top', 'y_top', 'x_bottom', 'y_bottom',
       'label'],
      dtype='object')


,Unnamed: 0,Image,x_top,y_top,x_bottom,y_bottom,label
0,0,image_0001.jpg,30,137,49,349,Airplanes
1,0,image_0002.jpg,35,153,59,342,Airplanes
2,0,image_0003.jpg,36,135,47,331,Airplanes
3,0,image_0004.jpg,24,141,47,342,Airplanes
4,0,image_0005.jpg,18,146,48,339,Airplanes


 2. Convertir a formato COCO (x, y, width, height)

In [6]:
# Función para corregir etiquetas (coordenadas)
def corners_to_coco(x_top, y_top, x_bottom, y_bottom):
    x = x_bottom
    y = x_top
    width = y_bottom - x_bottom
    height = y_top - x_top
    return [x, y, width, height]

# Recalcular las coordenadas usando la función corregida
df[['x', 'y', 'width', 'height']] = df.apply(
    lambda row: pd.Series(corners_to_coco(row['x_top'], row['y_top'], row['x_bottom'], row['y_bottom'])),
    axis=1
)

# Coordenadas finales
df['x2'] = df['x'] + df['width']
df['y2'] = df['y'] + df['height']

# Normalizar para entrenamiento
IMAGE_WIDTH = 512
IMAGE_HEIGHT = 512
df['x_norm'] = df['x'] / IMAGE_WIDTH
df['y_norm'] = df['y'] / IMAGE_HEIGHT
df['x2_norm'] = df['x2'] / IMAGE_WIDTH
df['y2_norm'] = df['y2'] / IMAGE_HEIGHT

# Vector para regresión
df['bbox_norm'] = df[['x_norm', 'y_norm', 'x2_norm', 'y2_norm']].values.tolist()

# Agregar ruta de imagen
df['image_path'] = df['Image'].apply(lambda x: f"airplanes/airplanes{x}")
df.head()

,Unnamed: 0,Image,x_top,y_top,x_bottom,y_bottom,label,x,y,width,height,x2,y2,x_norm,y_norm,x2_norm,y2_norm,bbox_norm,image_path
0,0,image_0001.jpg,30,137,49,349,Airplanes,49,30,300,107,349,137,0.095703,0.058594,0.681641,0.267578,"[0.095703125, 0.05859375, 0.681640625, 0.26757...",airplanes/airplanesimage_0001.jpg
1,0,image_0002.jpg,35,153,59,342,Airplanes,59,35,283,118,342,153,0.115234,0.068359,0.667969,0.298828,"[0.115234375, 0.068359375, 0.66796875, 0.29882...",airplanes/airplanesimage_0002.jpg
2,0,image_0003.jpg,36,135,47,331,Airplanes,47,36,284,99,331,135,0.091797,0.070312,0.646484,0.263672,"[0.091796875, 0.0703125, 0.646484375, 0.263671...",airplanes/airplanesimage_0003.jpg
3,0,image_0004.jpg,24,141,47,342,Airplanes,47,24,295,117,342,141,0.091797,0.046875,0.667969,0.275391,"[0.091796875, 0.046875, 0.66796875, 0.275390625]",airplanes/airplanesimage_0004.jpg
4,0,image_0005.jpg,18,146,48,339,Airplanes,48,18,291,128,339,146,0.093750,0.035156,0.662109,0.285156,"[0.09375, 0.03515625, 0.662109375, 0.28515625]",airplanes/airplanesimage_0005.jpg


In [7]:
import matplotlib.pyplot as plt
import cv2

def show_image_with_box(image_path, x1, y1, x2, y2):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found or cannot be read: {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.gca().add_patch(
        plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='red', facecolor='none', linewidth=2)
    )
    plt.axis('off')
    plt.show()

# Mostrar la primera imagen
row = df.iloc[0]
show_image_with_box(row['image_path'], row['x'], row['y'], row['x2'], row['y2'])


[ WARN:0@68.930] global loadsave.cpp:268 findDecoder imread_('airplanes/airplanesimage_0001.jpg'): can't open/read file: check file path/integrity


FileNotFoundError: Image not found or cannot be read: airplanes/airplanesimage_0001.jpg

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class AirplaneDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row['image_path']).convert('RGB')
        bbox = torch.tensor(row['bbox_norm'], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, bbox

# Aumentación y normalización
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])

# Crear dataset y dataloader
dataset = AirplaneDataset(df, transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
import torch.nn as nn
from torchvision.models import resnet18, resnet34

# Modelo con ResNet18 como backbone
class ResNetBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        # base = resnet18(pretrained=True)
        base = resnet34(pretrained=True)
        self.features = nn.Sequential(*list(base.children())[:-1])  # quitar capa final
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 4),
            nn.Sigmoid()  # Para mantener coordenadas entre 0 y 1
        )

    def forward(self, x):
        x = self.features(x)
        return self.regressor(x)

model = ResNetBackbone().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()


In [ ]:
def compute_iou(box1, box2):
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = max((xi2 - xi1), 0) * max((yi2 - yi1), 0)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

In [ ]:
# Entrenamiento
best_model = None
best_iou = 0.0
iou_per_epoch = []
EPOCHS = 15

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, targets in dataloader:
        images, targets = images.to(device), targets.to(device)
        outputs = model(images)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"🔁 Epoch {epoch+1}/{EPOCHS} - Loss promedio: {avg_loss:.4f}")

In [ ]:
import numpy as np
import copy
 
# Evaluar IoU en 20 muestras
model.eval()
ious = []
for i in range(20):
    image, target = dataset[i]
    image = image.unsqueeze(0).to(device)
    pred = model(image).detach().cpu().numpy().flatten()
    true = target.numpy().flatten()
    pred_box = [pred[0]*512, pred[1]*512, pred[2]*512, pred[3]*512]
    true_box = [true[0]*512, true[1]*512, true[2]*512, true[3]*512]
    ious.append(compute_iou(pred_box, true_box))

avg_iou = np.mean(ious)
iou_per_epoch.append(avg_iou)
print(f"🔍 IoU promedio: {avg_iou:.3f}")

if avg_iou > best_iou:
    best_model = copy.deepcopy(model.state_dict())
    best_iou = avg_iou
    print("✅ Nuevo mejor modelo guardado.")

torch.save(best_model, "mejor_modelo_bbox.pth")
print("📦 Modelo guardado.")

In [ ]:
# box verde en la imagen: Ground Truth (bounding box real del avión)
# box rojo en la imagen: Predicción del modelo (bounding box predicha)
import matplotlib.pyplot as plt
import cv2
import random

def visualize_predictions(model, dataset, num_images=5):
    model.eval()
    indices = random.sample(range(len(dataset)), num_images)
    fig, axes = plt.subplots(1, num_images, figsize=(20, 5))

    for i, idx in enumerate(indices):
        image, true_bbox = dataset[idx]
        image_input = image.unsqueeze(0).to(device)
        pred_bbox = model(image_input).detach().cpu().numpy().flatten()
        true = [int(x * 512) for x in true_bbox]
        pred = [int(x * 512) for x in pred_bbox]
        image_path = df.iloc[idx]['image_path']
        img = cv2.imread(image_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.rectangle(img, (true[0], true[1]), (true[2], true[3]), (0, 255, 0), 2)
        cv2.rectangle(img, (pred[0], pred[1]), (pred[2], pred[3]), (255, 0, 0), 2)
        axes[i].imshow(img)
        axes[i].set_title(f'Img {idx}')
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

visualize_predictions(model, dataset, num_images=5)